In [1]:
import torch
import torch.nn as nn

class SimpleHymbaWithMetaTokens(nn.Module):
    """
    메타 토큰 처리 흐름을 명확히 보여주기 위한 간소화된 모델
    """
    def __init__(self, config):
        super().__init__()
        self.config = config
        self.d_model = config['d_model']
        self.n_meta_tokens = config['n_meta_tokens']

        # 학습 가능한 메타 토큰 파라미터 생성
        # Shape: (1, n_meta_tokens, d_model)
        self.meta_tokens = nn.Parameter(torch.randn(1, self.n_meta_tokens, self.d_model))
        
        # 일반 토큰을 위한 임베딩 레이어
        self.token_embeddings = nn.Embedding(config['vocab_size'], self.d_model)
        
        # 모델의 내부 레이어들을 단순화한 예시
        self.internal_layers = nn.Sequential(
            nn.Linear(self.d_model, self.d_model),
            nn.ReLU(),
            nn.Linear(self.d_model, self.d_model)
        )

        # 최종 출력을 위한 언어 모델 헤드
        self.lm_head = nn.Linear(self.d_model, config['vocab_size'])

    def forward(self, tokens: torch.Tensor):
        batch_size, seq_len = tokens.shape
        print(f"--- 1. 입력 단계 ---")
        print(f"입력 토큰 Shape: {tokens.shape}")

        # 토큰을 임베딩으로 변환
        input_embeds = self.token_embeddings(tokens)
        print(f"입력 임베딩 Shape: {input_embeds.shape}")

        # 메타 토큰을 배치 사이즈만큼 복제
        meta_embeds = self.meta_tokens.expand(batch_size, -1, -1)
        print(f"복제된 메타 토큰 Shape: {meta_embeds.shape}")

        # ===> 시퀀스 확장 <===
        combined_embeds = torch.cat([meta_embeds, input_embeds], dim=1)
        print(f"▶ 메타 토큰 결합 후 확장된 시퀀스 Shape: {combined_embeds.shape}\n")
        
        print(f"--- 2. 내부 처리 단계 ---")
        # 모델의 내부 레이어 통과 (확장된 시퀀스로 처리)
        processed_output = self.internal_layers(combined_embeds)
        print(f"내부 레이어 통과 후 Shape: {processed_output.shape}\n")

        print(f"--- 3. 출력 슬라이싱 단계 ---")
        # ===> 시퀀스 슬라이싱 <===
        # 최종 로짓 계산을 위해 메타 토큰에 해당하는 부분을 잘라냄
        # Shape: (batch_size, seq_len, d_model)
        output_for_lm_head = processed_output[:, self.n_meta_tokens:, :]
        print(f"▶ 메타 토큰 슬라이싱 후 원래 시퀀스 Shape: {output_for_lm_head.shape}")

        # 최종 로짓 계산
        final_logits = self.lm_head(output_for_lm_head)
        print(f"최종 로짓 Shape: {final_logits.shape}")

        return final_logits

# --- 테스트 실행 ---
if __name__ == '__main__':
    # 모델 설정
    config = {
        'vocab_size': 32000,
        'd_model': 256,
        'n_meta_tokens': 4,
    }

    # 모델 초기화
    model = SimpleHymbaWithMetaTokens(config)
    
    # 더미 입력 데이터 생성
    dummy_input = torch.randint(0, config['vocab_size'], (2, 512)) # (batch_size=2, seq_len=512)
    
    # 모델 실행
    with torch.no_grad():
        output = model(dummy_input)

    print("\n" + "="*40)
    print("최종 요약:")
    print(f"입력 시퀀스 길이: {dummy_input.shape[1]}")
    print(f"내부 처리 시퀀스 길이: {dummy_input.shape[1] + config['n_meta_tokens']}")
    print(f"최종 출력 시퀀스 길이: {output.shape[1]}")
    print("="*40)


--- 1. 입력 단계 ---
입력 토큰 Shape: torch.Size([2, 512])
입력 임베딩 Shape: torch.Size([2, 512, 256])
복제된 메타 토큰 Shape: torch.Size([2, 4, 256])
▶ 메타 토큰 결합 후 확장된 시퀀스 Shape: torch.Size([2, 516, 256])

--- 2. 내부 처리 단계 ---
내부 레이어 통과 후 Shape: torch.Size([2, 516, 256])

--- 3. 출력 슬라이싱 단계 ---
▶ 메타 토큰 슬라이싱 후 원래 시퀀스 Shape: torch.Size([2, 512, 256])
최종 로짓 Shape: torch.Size([2, 512, 32000])

최종 요약:
입력 시퀀스 길이: 512
내부 처리 시퀀스 길이: 516
최종 출력 시퀀스 길이: 512
